# Assignment 4: Pipelines and Hyperparameter Tuning (32 total marks)
### Due: November 22 at 11:59pm

### Name: 

### In this assignment, you will be putting together everything you have learned so far. You will need to find your own dataset, do all the appropriate preprocessing, test different supervised learning models and evaluate the results. More details for each step can be found below.

### You will also be asked to describe the process by which you came up with the code. More details can be found below. Please cite any websites or AI tools that you used to help you with this assignment.

## Import Libraries

In [2]:
import numpy as np
import pandas as pd

## Step 1: Data Input (4 marks)

Import the dataset you will be using. You can download the dataset onto your computer and read it in using pandas, or download it directly from the website. Answer the questions below about the dataset you selected. 

To find a dataset, you can use the resources listed in the notes. The dataset can be numerical, categorical, text-based or mixed. If you want help finding a particular dataset related to your interests, please email the instructor.

**You cannot use a dataset that was used for a previous assignment or in class**

In [7]:
# Import dataset (1 mark)
total_data = pd.read_csv('nba_2022_2023.csv')

### Questions (3 marks)

1. (1 mark) What is the source of your dataset?
1. (1 mark) Why did you pick this particular dataset?  
1. (1 mark) Was there anything challenging about finding a dataset that you wanted to use?  
 
 *ANSWER HERE*

1.  Bryan Weather Chung, October 12/2023, "NBA Player Stats Dataset for the 2022-2023", Kaggle. [Online].    Available : https://www.kaggle.com/datasets/bryanchungweather/nba-players-data-2022-2023/data    
  
  
2.   I'm a big fan of the NBA, and I thought it would be cool to make a machine learning model on a dataset with NBA players statistics. My goal is to make a target vector of points per game, using the rest of the columns as features. I may drop some features such as "Player" because we don't need to know a players name predict a players points, just the players statistics.  
  
  
3.  Nope, I just went to kaggle found the most trending datasets and found this was one of them.


## Step 2: Data Processing (5 marks)

The next step is to process your data. Implement the following steps as needed.

In [18]:
# Our goal is to make a machine learning model that will predict a NBA players points per game. Let us check if 
# we have any columns that would be irrelevant to determing a players points per game.

pd.set_option('display.max_columns', None)
total_data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Precious Achiuwa,C,23,TOR,55,12,20.7,3.6,7.3,0.485,0.5,2.0,0.269,3.0,5.4,0.564,0.521,1.6,2.3,0.702,1.8,4.1,6.0,0.9,0.6,0.5,1.1,1.9,9.2
1,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,0.597,0.0,0.0,0.000,3.7,6.2,0.599,0.597,1.1,3.1,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
2,Bam Adebayo,C,25,MIA,75,75,34.6,8.0,14.9,0.540,0.0,0.2,0.083,8.0,14.7,0.545,0.541,4.3,5.4,0.806,2.5,6.7,9.2,3.2,1.2,0.8,2.5,2.8,20.4
3,Ochai Agbaji,SG,22,UTA,59,22,20.5,2.8,6.5,0.427,1.4,3.9,0.355,1.4,2.7,0.532,0.532,0.9,1.2,0.812,0.7,1.3,2.1,1.1,0.3,0.3,0.7,1.7,7.9
4,Santi Aldama,PF,22,MEM,77,20,21.8,3.2,6.8,0.470,1.2,3.5,0.353,2.0,3.4,0.591,0.560,1.4,1.9,0.750,1.1,3.7,4.8,1.3,0.6,0.6,0.8,1.9,9.0


All these columns are relevant for determing a players points per game. For example, the "Player" column names a 
player. We could use a players past results to help determine a players future results. The "Tm" column names a 
team. This is very important because many teams have different coaches and philosphies on how basketball should
be played.

In [26]:
# Clean data (if needed)
# Check for any null values
null_values = total_data.isnull().sum()
null_values

Player     0
Pos        0
Age        0
Tm         0
G          0
GS         0
MP         0
FG         0
FGA        0
FG%        3
3P         0
3PA        0
3P%       24
2P         0
2PA        0
2P%        7
eFG%       3
FT         0
FTA        0
FT%       37
ORB        0
DRB        0
TRB        0
AST        0
STL        0
BLK        0
TOV        0
PF         0
PTS        0
dtype: int64

In [27]:
# Replace null values with mean of the columns
total_data = total_data.fillna(total_data.mean(numeric_only=True))

In [28]:
#See if any null values remain
null_values = total_data.isnull().sum()
null_values

Player    0
Pos       0
Age       0
Tm        0
G         0
GS        0
MP        0
FG        0
FGA       0
FG%       0
3P        0
3PA       0
3P%       0
2P        0
2PA       0
2P%       0
eFG%      0
FT        0
FTA       0
FT%       0
ORB       0
DRB       0
TRB       0
AST       0
STL       0
BLK       0
TOV       0
PF        0
PTS       0
dtype: int64

In [51]:
# Implement preprocessing steps. Remember to use ColumnTransformer if more than one preprocessing method is needed

# We will use columns transformer to apply One-Hot-Encoding to columns with categorical variables.These columns are 
# "Player", "Pos" and "Tm" refering to a players name, their position, and their team. We will also scale the data 
# in case we use models that are senstivie to unscaled data.

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split


columns_to_scale = total_data.columns.difference(['Player', 'Pos', 'Tm', 'PTS'])
columns_to_encode = ['Player', 'Pos', 'Tm']

ct = ColumnTransformer([
        ("scaling", StandardScaler(), columns_to_scale),
        ("onehot", OneHotEncoder(sparse_output=False, handle_unknown ='ignore'), columns_to_encode)
        ])

X = total_data.drop("PTS", axis=1)
y = total_data["PTS"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

### Questions (2 marks)

1. (1 mark) Were there any missing/null values in your dataset? If yes, how did you replace them and why? If no, describe how you would've replaced them and why.
2. (1 mark) What type of data do you have? What preprocessing methods would you have to apply based on your data types?

*ANSWER HERE*

1. Yes, a few null values appeared in the dataset. I replaced them by the mean of the columns. Since the missing data was all from numerical columns, I could easily fill them in with the mean of the their respective columns. Also, the missing data was all statistics of a player, it would make much more sense to fill in the missing data with the mean of the column instead of backfilling or front filling. This is because all the rows are unrelated, but the column is related.  
  
  
2. My data is mostly numerical data describing the statistics of a nba player in the 2022 - 2023 season. Their are some categorical values such as "Player", "Pos", and "Tm". These three columns describe a players name, their position, and their team. The preprocessing methods I used are One-Hot-Encoding for columns with categorical values I just decribed. I also scaled the data in case I choose to use a model that is sensitive to unscaled data.

## Step 3: Implement Machine Learning Model (11 marks)

In this section, you will implement three different supervised learning models (one linear and two non-linear) of your choice. You will use a pipeline to help you decide which model and hyperparameters work best. It is up to you to select what models to use and what hyperparameters to test. You can use the class examples for guidance. You must print out the best model parameters and results after the grid search.

In [95]:
# Implement pipeline and grid search here. Can add more code blocks if necessary

# We will use two non-linear models, RandomForest and SVM. We will use one linear model ridge regression.First, we
# create pipeline for all 3 models, select grid paramaters, than apply GridSearchCV

from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Random Forest Pipeline and parameter grid for Random Forest
rf_pipeline = Pipeline([ ('preprocessor', ct), ('model', RandomForestRegressor()) ])

rf_param_grid = { 
        'model__n_estimators' : [5, 10, 15], 
        'model__max_depth': [5, 10, 15]
        }

#SVM Pipeline and parameter grid for SVM
svm_pipeline = Pipeline([ ('preprocessor', ct), ('model', SVR()) ])

svm_param_grid = {
        'model__C': [0.1, 1, 10],
        'model__kernel': ['poly', 'rbf'],
        'model__gamma': [0.1, 1, 10]
        }

# Ridge regression pipeline and parameter grid for ridge
Lasso_pipeline = Pipeline([ ('preprocessor', ct), ('model', Lasso()) ])

Lasso_param_grid = {
        'model__alpha': [0.1, 1, 10],
        'model__max_iter': [1000, 5000, 10000]
        }

# Apply GridSearchCV to all models and their respective parameter grids
rf_grid_search = GridSearchCV(rf_pipeline, rf_param_grid, cv = 5, n_jobs = -1)
svm_grid_search = GridSearchCV(svm_pipeline, svm_param_grid, cv = 5, n_jobs = -1)
Lasso_grid_search = GridSearchCV(Lasso_pipeline, Lasso_param_grid, cv = 5, n_jobs = -1)

# Fit the models
rf_grid_search.fit(X_train, y_train)
svm_grid_search.fit(X_train, y_train)
Lasso_grid_search.fit(X_train, y_train)


#Results for all three models - mean_test_score
rf_results = rf_grid_search.cv_results_['mean_test_score']
svm_results = svm_grid_search.cv_results_['mean_test_score']
Lasso_results = Lasso_grid_search.cv_results_['mean_test_score']

# Print Best Parameters and Results
print("Please Note: I choose a small amount of parameters since my laptop was taking a while to execute a lot of parameters")
print("\nBest Model Parameters for RandomForest, SVM, and Lasso\n")
print("Random Forest Results:")
print("Best Parameters:", rf_grid_search.best_params_)
print("Grid Search Best Results:", rf_results[rf_grid_search.best_index_])

print("\nSVM Results:")
print("Best Parameters:", svm_grid_search.best_params_)
print("Grid Search Best Results:", svm_results[svm_grid_search.best_index_])

print("\nLasso Results:")
print("Best Parameters:", Lasso_grid_search.best_params_)
print("Grid Search Best Results:", Lasso_results[Lasso_grid_search.best_index_])


Please Note: I choose a small amount of parameters since my laptop was taking a while to execute a lot of parameters

Best Model Parameters for RandomForest, SVM, and Lasso

Random Forest Results:
Best Parameters: {'model__max_depth': 15, 'model__n_estimators': 10}
Grid Search Best Results: 0.9872112796119143

SVM Results:
Best Parameters: {'model__C': 0.1, 'model__gamma': 0.1, 'model__kernel': 'poly'}
Grid Search Best Results: 0.8007287887722742

Lasso Results:
Best Parameters: {'model__alpha': 0.1, 'model__max_iter': 1000}
Grid Search Best Results: 0.9994697072325331


### Questions (5 marks)

1. (1 mark) Do you need regression or classification models for your dataset?
1. (2 marks) Which models did you select for testing and why?
1. (2 marks) Which model worked the best? Does this make sense based on the theory discussed in the course and the context of your dataset?

*ANSWER HERE*

1. Regression, my targets vector is 'PTS' which represents points a player gets per game, these are all numerical values.  
  
  
2. I used Random Forest, SVM, and ridge models. The reason I used Random Forest was because from my past experiences, it appears to be one of the most powerful models and relatively quick as well. Since my One-Hot-Encoding made this dataset very high deminesional, I thought this would be a good dataset to see the limitations of Random Forest. I also chose this model because gradient boosting machines requires an iterative approach, and that would take much longer to execute.      

    I used a SVM model as well. SVM models work well with high dimensional data, so I thought it would be a good fit for this dataset. Since the number of features exploded with One-Hot-Encoding. Another reason I this model was because I wanted to get a better understanding on how the pipeline class works, and since SVM model requires scaling, it was a good choice to improve my understanding of the pipeline. I also wanted to see how changing C and gamma would impact the results.  
    
    For my linear model, I choose lasso regression. The reason I choose lasso was because I knew that many features would be irrelevant when training my model. So it was important, that we could apply L1 regularization and put some coefficants for some features to exactly 0.  
      
3. The best model was the lasso model. It had a R2 score of 0.9995. Much better than Random Forest which had a R2 score of 0.98721 and SVM which had a R2 score of 0.80073. This does make sense since linear models typically do well when the number of features is large compared to number of samples.   
  
    Random Forest might have done worse than my Lasso model because the dataset might not have been large enough. We know Random Forest does better with large datasets but this dataset has only 680 samples. Also Random Forest considers all features, where as Lasso only considers the relevant features. 
    
    Same thing can be said about SVM, where SVM considers all features while Lasso only considers the most important features. We also might have chose the wrong type of scaling for this dataset, maybe MinMaxScalar would have been better for this dataset.  
      
    It is important to note for this dataset, we have a linear relationships between many features and the target vector. For example, feature "FG%" and our target vector "PTS" share a linear relationship. Since our lasso model works best with linear models, this is probably the biggest reason Lasso model did the best.



## Step 4: Validate Model (6 marks)

Use the testing set to calculate the testing accuracy for the best model determined in Step 3.

In [100]:
# Calculate testing accuracy (1 mark)
# The best model was the Lasso Model

lasso_result = Lasso_grid_search.score(X_test, y_test)
print(lasso_result)

0.9995043866770325



### Questions (5 marks)

1. (1 mark) Which accuracy metric did you choose? 
1. (1 mark) How do these results compare to those in part 3? Did this model generalize well?
1. (3 marks) Based on your results and the context of your dataset, did the best model perform "well enough" to be used out in the real-world? Why or why not? Do you have any suggestions for how you could improve this analysis?

*ANSWER HERE*

1. I used the R2 accuracy metric.  
  
2.  The results are remarkably similar. We got a result of 0.9995 on the best result from GridSearchCV. Here, for our validation test, we got a R2 score of 0.9995 as well. In fact, it's a bit higher if we don't round it to the 4th decimal place. Yes the model generalized extremely well.
  
3. Yes, the model peformed exceptionally, and it is certainly able to be used in the real world. A R2 score of 0.9995 is nearly perfect and I am sure it would work well in the real world if released. This model also does not evaluate anything critical such as health of a patient or future floods. It is a simple model that simply evaluates how much pts per game a NBA player would have in a season based on some statistics. It will have no major harmful impact on the real world. I do think we can improve the results using ElasticNet model. This will allow us to use L1 and L2 regularization to improve our scores. I could also check a wider range for alpha instead of 0.1 to 10. Unfortunately, It does take a lot of time to execute.

## Process Description (4 marks)
Please describe the process you used to create your code. Cite any websites or generative AI tools used. You can use the following questions as guidance:
1. Where did you source your code?
1. In what order did you complete the steps?
1. If you used generative AI, what prompts did you use? Did you need to modify the code at all? Why or why not?
1. Did you have any challenges? If yes, what were they? If not, what helped you to be successful?

*DESCRIBE YOUR PROCESS HERE*

1. Bryan Weather Chung, October 12/2023, "NBA Player Stats Dataset for the 2022-2023", Kaggle. [Online].    Available : https://www.kaggle.com/datasets/bryanchungweather/nba-players-data-2022-2023/data  
  
2. First I found the dataset that I was interested in. I read in the data using pandas. I started data processing, by checking all columns for categorical values. I filled in any missing values with the mean of that respective column. I loaded scaling and One-Hot-Encoding into the columnTransformer class for our pipeline class to implement. Afterwords, I split the data into my target vector and feature matrix, and used test_train_split to split the data into testing and training portions. I than created 3 pipeline classes with 3 different models. I then used GridSearchCV to create find the best parameters of each model. Once the best model was found with the best training score, we used the testing set to check our validation score. Then, we finally reported our validation score.  
  
3. Yes I used chatGPT for this assignment. I prompted chatGPT to for what was stored in "cv_results_" of "GridSearchCV". I was having trouble finding the testing scores for the models and asked chatGPT to explain "cv_results_" and what was stored in that attribute. I also asked chatGPT how to make "GridSearchCV" faster as well because it was taking so long. So I added "n_jobs = -1" to use all available CPU's. I had to modify the code to add "n_jobs" and "mean_test_score" to find the results of GridSearchCV execution.  
  
4. Yes, I struggled with GridSearchCV class. I had trouble with how long it was taking to execute. I also had trouble understanding the cv_results_ attribute of GridSearchCV. Their were many variables in that attribute and I struggled understanding what each one meant.

## Reflection (2 marks)
Include a sentence or two about:
- what you liked or disliked,
- found interesting, confusing, challenging, motivating
while working on this assignment.


*ADD YOUR THOUGHTS HERE*

I liked learning about pipelines and GridSearchCV. Very powerful tools that allow us to find the best models with the best parameters. I disliked how long it took to execute GridSearchCV. What I found interesting and confusing was the cv_results_ attribute of GridSearchCV. It holds a lot of different information that can be used in many differeny ways. I also found using columntransformers as a preprocessing step in pipelines is a very powerful and convenient way to preprocess data.